In [24]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Define the path to the directory
directory_path = r'F:\Underwater_Image\Sea Animals\archive'

# Initialize empty lists for images and labels
X_train = []
Y_train = []

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Loop through subdirectories (classes)
for class_folder in os.listdir(directory_path):
    class_path = os.path.join(directory_path, class_folder)
    class_label = class_folder
    
    # Encode class labels
    encoded_label = label_encoder.fit_transform([class_label])[0]

    for image_file in os.listdir(class_path):
        if image_file.endswith('.jpg'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((32, 32))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_train.append(img_array)
            Y_train.append(encoded_label)

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')


X_train shape: (13666, 32, 32, 3)
Y_train shape: (13666,)


In [25]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Convert labels to categorical format with 23 classes
Y_train = to_categorical(Y_train, num_classes=23)

# Perform train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

# Check the shapes
print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'Y_test shape: {Y_test.shape}')

X_train shape: (10932, 32, 32, 3)
Y_train shape: (10932, 23)
X_test shape: (2734, 32, 32, 3)
Y_test shape: (2734, 23)


In [26]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

# Create the ResNet model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    
# Add the top layers for classification
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(23, activation='softmax'))  # Change the number of units and activation
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=64, epochs=2, validation_split = .1)

Epoch 1/2
154/154 [==============================] - 30s 117ms/step - loss: 0.0743 - accuracy: 0.9861 - val_loss: 0.0104 - val_accuracy: 1.0000
Epoch 2/2
154/154 [==============================] - 13s 86ms/step - loss: 5.1509e-06 - accuracy: 1.0000 - val_loss: 3.8883e-05 - val_accuracy: 1.0000


In [27]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
batch_size = 64
# Evaluate the model on the test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))
test_dataset = test_dataset.batch(batch_size)

test_loss, test_accuracy = model.evaluate(test_dataset)

# Make predictions on the test dataset
predictions = model.predict(test_dataset)

Y_test = np.argmax(Y_test , axis=1)
# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(Y_test, predicted_labels)

# Calculate F1 score
f1 = f1_score(Y_test, predicted_labels, average='weighted')

# Calculate confusion matrix
conf_matrix = confusion_matrix(Y_test, predicted_labels)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")


43/43 [==============================] - 2s 42ms/step - loss: 3.8709e-05 - accuracy: 1.0000
Test Loss: 3.870879299938679e-05
Test Accuracy: 1.0
Accuracy: 1.0
F1 Score: 1.0
